In [1]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")


2023-04-25 13:52:23.584207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 13:52:24.908178: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import torch

# Evaluation du modèle

In [4]:
from datasets import load_dataset, load_metric

In [5]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import os
import json
import s3fs

## Charger les données

In [6]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "mberthe/narval"
fs.ls(BUCKET)
BUCKET = "mberthe/narval"
FILE_KEY_S3 = "camemBERT/answers(2).json"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    f1=json.load(file_in)

In [1]:
import numpy as np # linear algebra
from subprocess import check_output

In [8]:
df=pd.DataFrame(f1["data"])
df.head()

,paragraphs
0,[{'qas': [{'question': 'quelle est la valeur ...
1,[{'qas': [{'question': 'quelle est la valeur d...
2,[{'qas': [{'question': 'Quelle est la valeur d...
3,[{'qas': [{'question': 'quelle est la valeur d...
4,[{'qas': [{'question': 'Quelle est la valeur d...


In [9]:
input_file_path = FILE_PATH_S3
record_path = ['data','paragraphs','qas','answers']
rpqs = pd.json_normalize(f1, record_path =record_path,meta=[
        ['data', 'paragraphs','context'], 
        ['data', 'paragraphs', 'qas','id'],
        ['data', 'paragraphs', 'qas','question']])

In [10]:
rpqs=rpqs.rename(columns={"data.paragraphs.context":"context","data.paragraphs.qas.id":"id","data.paragraphs.qas.question":"question"})

In [11]:
rpqs

,answer_id,document_id,question_id,text,answer_start,answer_end,answer_category,context,id,question
0,726620,1364517,665706,"99,8",975,979,None,Rapport sur le prix et la qualité de l’eau pot...,665706,quelle est la valeur du taux de conformité de...
1,726622,1364517,665707,100 %,1009,1014,None,Rapport sur le prix et la qualité de l’eau pot...,665707,quelle est la valeur du taux de conformité des...
2,726502,1364517,665606,"99,8 %",974,982,None,Rapport sur le prix et la qualité de l’eau pot...,665606,Quelle est la valeur de P101.1?
3,726504,1364517,665608,100 %,1008,1014,None,Rapport sur le prix et la qualité de l’eau pot...,665608,Quelle est la valeur de P101.2 ?
4,726505,1364517,665609,107,4037,4041,None,Rapport sur le prix et la qualité de l’eau pot...,665609,Quelle est la valeur de P103.2B
...,...,...,...,...,...,...,...,...,...,...
121,727136,1364611,666176,"2,11",1695,1699,None,\n \n Les indicateurs de performances \n \nD...,666176,Quel est le taux d'impayés sur les factures d'...
122,727138,1364611,666178,"0,20",1778,1783,None,\n \n Les indicateurs de performances \n \nD...,666178,Quel est le taux de réclamation ?
123,727139,1364611,666179,"0,20",1778,1782,None,\n \n Les indicateurs de performances \n \nD...,666179,Quelle est la valeur de P258.1?
124,727150,1364619,666190,"0,014",827,833,None,19 5 Actions de solidarité et de coopération \...,666190,Quelle est la valeur de P109.0?


In [12]:
rpqs['question'][0]

'quelle est la valeur  du taux de conformité des prélèvements sur les eaux distribuées réalisés au titre du contrôle sanitaire par rapport aux limites de qualité pour ce qui concerne la microbiologie'

In [13]:
import collections

### Créer un texte de la forme: Contexte: "..." Question : "..."

In [14]:
# making copy of question column
new = rpqs["question"].copy()
# concatenating team with name column
# overwriting name column
rpqs["context"]= rpqs["context"].str.cat(new, sep =". Question: ")
rpqs['context']= "Contexte: " + rpqs['context']
# display
#rpqs['context'][0]


### On retire les \n

In [15]:
def n(text):
        regex="\n"
        return re.sub(regex, " ", text)

In [16]:
import re
for i in rpqs.index:
    rpqs['context'][i]=n(rpqs['context'][i])

### Prédiction sur 1 question

In [18]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")

input_text = rpqs['context'][0]
input_ids = tokenizer(input_text, max_length=510,
        truncation="only_second",
        padding="max_length",return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


/opt/mamba/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad>- - - - - - - - -


### Prédictions toutes les questions

In [20]:
answer_pred=[]
for i in rpqs.index:
    input_text = rpqs['context'][i]
    input_ids = tokenizer(input_text, max_length=510,
        truncation="only_second",
        padding="max_length",return_tensors="pt").input_ids

    outputs = model.generate(input_ids)
    answer_pred.append(tokenizer.decode(outputs[0]))

In [22]:
answer_pred

['<pad>- - - - - - - - -',
 '<pad>- - - - - - - - -',
 '<pad> 0</s>',
 '<pad> 0</s>',
 "<pad> <unk>l'heure actuelle, à l'heure actuelle, à",
 '<pad> 79,9 %</s>',
 '<pad> TOTAL</s>',
 "<pad> <unk> l'heure de l'année à l'autre</s>",
 '<pad> VP.056/parenright.case</s>',
 '<pad> D203.0/parenright.case</s>',
 '<pad>/parenleft.caseD203.0/parenright.case /',
 '<pad> 0.3 %</s>',
 '<pad>quotient, expressed in percent, of the average over 5 years of the network renewe',
 '<pad>quotient, expressed in percent, of the average over 5 years of the network renewe',
 '<pad> 4 points noirs par 100 km</s>',
 '<pad> 100 % 100 % 100 % 100 % 100 %</s>',
 '<pad> 100 % 100 % 100 % 100 % 100 % 100 %</s>',
 '<pad> 1,0 1,3 1,2 1,1 1,5</s>',
 '<pad>0.7 % 0,7 % 0,7 % 0,8 % 0,7',
 '<pad> 73,18% %</s>',
 '<pad>11 203 Indicateurs de performance P104.3 Rendement de distribution 73,',
 '<pad> 9,96 m3/km/j</s>',
 '<pad> 73,18% %</s>',
 '<pad> 9,28 m3/km/j</s>',
 '<pad>Taux moyen de renouvellement des réseaux d’eau potab

In [23]:
rpqs['answer_pred']=answer_pred
rpqs[['text','answer_pred']].sample(15)

,text,answer_pred
40,80 %,<pad> 80 %</s>
12,100 %,"<pad>quotient, expressed in percent, of the av..."
18,"0,7","<pad>0.7 % 0,7 % 0,7 % 0,8 % 0,7"
110,"0,002",<pad> 100 %</s>
45,100%,<pad> 0</s>
105,3 893,<pad> 3 893 T de MS</s>
106,"3,33",<pad> D203.0</s>
122,"0,20",<pad> 0.20 pour 1000 abonnés</s>
36,"0,0010 €","<pad>01 - Rennes-St Jacques 0,0010 € 0,0010 € 02"
125,"0,014",<pad> 014 €/m3</s>


### Enregistrer les prédictions

In [24]:
BUCKET_OUT = "mberthe/narval"
FILE_KEY_OUT_S3 = "camemBERT/pred_flan-base.csv"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w', encoding="utf-8") as file_out:
    rpqs.to_csv(file_out)

### Charger les prédictions

In [25]:
#load les predictions 
BUCKET = "mberthe/narval"
FILE_KEY_S3 = "camemBERT/pred_flan-base.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    rpqs = pd.read_csv(file_in)

## Normalisation et évaluation des données

In [26]:
def normalize_text(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    
    def no_unit(text):
        regex="m3"
        return re.sub(regex, " ", text)
    def no_exp(text):
        regex="m³"
        return re.sub(regex, " ", text)
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
   
    def white_space_fix(text):
        return " ".join(text.split())
    def no_date(text):
        date_extract_pattern = "[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}"
        return re.sub(date_extract_pattern,"",text)
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
   #on retire tout les caractère des réponses 
    def no_char(text):
        regex=r"([a-zA-Z])"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def indic(text):
        regex="(P\d\d\d\\.\d\s*)|(D\d\d\d\\.\d\s*)|(VP\\.\d\d\d)"
        return re.sub(regex," ", text)
    def pointov(text):
        regex="\."
        return re.sub(regex,",",text)
    return white_space_fix(pointov(no_punct(no_char(unidecode(comma(special_char(indic(no_exp(no_unit(no_date(s)))))))))))

In [27]:
def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

In [29]:
from unidecode import unidecode
import statistics

### Exact Match

In [30]:
em_list1=[]
for i in rpqs.index:
    truth=rpqs["text"][i]
    prediction1=rpqs["answer_pred"][i] 
    em1=compute_exact_match(prediction1,truth)
    em_list1.append(em1)
moyenne_em1=statistics.mean(em_list1)
#print('avant le traitement des predictions: em_moyen = 0,10')
print('après traitement des réponses : em_moyen=',moyenne_em1)

après traitement des réponses : em_moyen= 0.23809523809523808


### F1

In [31]:
def compute_f1(prediction, truth):
    pred_tokens =normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [32]:
f1_list=[]
for i in rpqs.index:
    truth=rpqs["text"][i]
    prediction=rpqs["answer_pred"][i] 
    f1=compute_f1(prediction,truth)
    #print(f1)
    f1_list.append(f1)
#print(f1_list)
moyenne_f1=statistics.mean(f1_list)
print('f1_moyen après traitement=',moyenne_f1)


f1_moyen après traitement= 0.3357331821617536


### Accuracy

In [33]:
def compute_acc(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    if len(common_tokens) == 0:
        return 0
    else :
        return 1

In [34]:
acc_list=[]
for i in rpqs.index:
    truth=rpqs["text"][i]
    #print(i,truth)
    prediction=rpqs["answer_pred"][i] 
    #print(i,prediction)
    acc=compute_acc(prediction,truth)
    #print("acc",acc)
    acc_list.append(acc)
#print(f1_list)
moyenne_acc=statistics.mean(acc_list)
print('acc_moyen après traitement=',moyenne_acc)

acc_moyen après traitement= 0.42857142857142855
